# M-stock for Rookie 생성 결과물 평가

## 라이브러리 불러오기

In [8]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import pandas as pd
import requests
import json
import re

## Clova Studio API를 사용한 실시간 텍스트 완성 요청 및 응답 처리

In [4]:
class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        response_text = ""
        with requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                           headers=headers, json=completion_request, stream=True) as r:
            for line in r.iter_lines():
                if line:
                    decoded_line = line.decode("utf-8")
                    if decoded_line.startswith('data:'):
                        try:
                            data = json.loads(decoded_line[5:])
                            if "message" in data:
                                response_text += data["message"]["content"]
                        except json.JSONDecodeError:
                            continue
        return response_text.strip()


## G-eval 정확성 평가

In [23]:
# -*- coding: utf-8 -*-
def eval_accuracy(origin,clova):
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='private_api_key',
        api_key_primary_val='private_api_key',
        request_id='a4b3de12-ea25-4dfd-a665-8369ef58fb5a'
    )
    
    input_text = "원문: " + origin +"요약: "+ clova

    preset_text = [
        {
            "role": "system",
            "content": """
            평가 기준:
            5점: 요약 내용이 원본에 포함된 정보와 논리적으로 일치합니다.
            1점: 요약 내용이 원본에 포함된 정보와 논리적으로 일치하지 않습니다.
            1점에서 5점 사이의 정수 점수: 요약 내용이 원본 정보에 따라 논리적으로 참인지 거짓인지 여부를 추가 정보 없이 결정할 수 없는 경우 1점을 사용합니다.
            
            작업 절차:
            1. 원본 정보를 철저히 읽습니다.
            2. 요약 내용을 철저히 읽고 원본 정보와 논리적으로 일치하는지 평가합니다.
            3. 평가 점수를 선택하여 기입합니다.
            
            ### 템플릿 (점수만 작성 only socre)
            정확성: {output}
            ###
            """
        },
        {
            "role": "user",
            "content": f"입력 : {input_text}"
        }
    ]

    request_data = {
        'messages': preset_text,
        'topP': 0.6,
        'topK': 0,
        'maxTokens': 300,
        'temperature': 0.5,
        'repeatPenalty': 2.0,
        'stopBefore': [],
        'includeAiFilters': True,
        'seed': 0
    }

    result = completion_executor.execute(request_data)
    return result

## 문장 나누기 및 Kobart 요약

요약문과 원문의 총 토큰값이 4096이 넘을 경우 문장을 나누고 Kobart로 요약한다

In [47]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# 토크나이저 및 모델 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

def split_text_into_paragraphs(text, num_paragraphs=3):
    # 텍스트를 문장 단위로 나눔
    sentences = text.split('. ')
    
    # 문장 수와 각 문단에 포함될 문장 수 계산
    total_sentences = len(sentences)
    sentences_per_paragraph = total_sentences // num_paragraphs
    
    paragraphs = []
    for i in range(num_paragraphs):
        start_index = i * sentences_per_paragraph
        # 마지막 문단은 남은 모든 문장을 포함하도록 함
        if i == num_paragraphs - 1:
            end_index = total_sentences
        else:
            end_index = (i + 1) * sentences_per_paragraph
        paragraph = '. '.join(sentences[start_index:end_index])
        # 마침표를 추가하여 문단을 완성
        if not paragraph.endswith('.'):
            paragraph += '.'
        paragraphs.append(paragraph)
    
    return paragraphs

def summarize_paragraphs(paragraphs):
    summaries = []
    for paragraph in paragraphs:
        text = paragraph.replace('\n', ' ')
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
        
        summary_ids = model.generate(torch.tensor([input_ids]), num_beams=4, max_length=512, eos_token_id=1)
        summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        summaries.append(summary)
    
    return ' '.join(summaries)

def summarize_text(text):
    if len(text) > 800:
        paragraphs = split_text_into_paragraphs(text)
        summary = summarize_paragraphs(paragraphs)
    else:
        summary = text
    return summary

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


## 토큰 수에 따른 평가 함수

In [13]:
def evaluate_text(origin,clova):
    # 토큰 수 계산
    tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
    tokenized_input1 = tokenizer.encode(origin)
    tokenized_input2 = tokenizer.encode(clova)
    token_count = len(tokenized_input1) + len(tokenized_input2)

    if token_count > 4096:
        # KoBART로 요약
        origin_summary = summarize_text(origin)
        print(f"요약된 텍스트: {origin_summary}")
        
        final_text = "원문: " + origin_summary +"요약: "+ clova
        # 요약된 텍스트로 정확성 평가
        accuracy_result = eval_accuracy(final_text)
    else:
        # 원문으로 바로 정확성 평가
        final_text = "원문: " + origin +"요약: "+ clova
        accuracy_result = eval_accuracy(final_text)
    
    return accuracy_result



## 중복 제거 함수

In [89]:
def remove_duplicates(text):
    pattern = r"정확성 : \s*\d"
    match = re.search(pattern, text)
    
    if match:
        first_occurrence_end = match.end()
        text = text[:first_occurrence_end]
    
    return text



## 적용 예시
경제 교육레터/해외 금융 뉴스레터

In [9]:
edu = pd.read_csv("/Users/kimminji/Downloads/미래에셋_요약&퀴즈.csv",encoding='cp949')
edu

,num,Title,Content,Summary
0,1,인구 감소 위기는 청소년 세대의 문제,Cover Story\n그래픽=김선우 한국경제신문 기자\n우리나라 인구 위기와 관련...,- 우리나라 인구 위기와 관련한 뉴스가 연일 쏟아짐\n- 대통령이 저출산고령사회위원...
1,2,"사회보험 지속가능성 '제로'…국민 부담, 어디까지 가능할까","(89) 사회보장제도 전망\n게티이미지뱅크\n정부가 국민연금, 건강보험 등 8대 사...","- 정부가 국민연금, 건강보험 등 8대 사회보험과 기초연금, 기초생활보장 등 공공부..."
2,3,"현대차, 60세 정년 후 2년간 '계속 고용'","생각하기와 글쓰기\n\n노사, '정년 연장 TF' 꾸려\n내년 상반기 합의안 도출\...",- 노사가 정년 연장 관련 TF를 꾸려 내년 상반기까지 정년 연장 관련 개선 방안을...
3,4,"원가 제때 반영 않는 전기·가스요금, 괜찮은가",일반 가정용 도시가스 도매 요금은 액화천연가스(LNG) 등 원자재 가격을 반영하는 ...,- 공공요금을 올리면 다른 상품이나 서비스 가격도 올라 고물가를 더 자극한다는 현...
4,5,중앙은행 무용론'…왜 나오는 걸까?,Cover Story\n그래픽=추덕영 한국경제신문 기자\n세계경제의 향방을 결정지을...,- 세계경제의 향방을 결정지을 변수로 미국 중앙은행(Fed)의 기준금리 인하 여부가...


In [16]:
# Content 열의 각 항목을 요약하여 Summary 열에 추가
edu['Summary_bart'] = edu['Content'].apply(summarize_text)

In [17]:
edu

,num,Title,Content,Summary,Summary_bart
0,1,인구 감소 위기는 청소년 세대의 문제,Cover Story\n그래픽=김선우 한국경제신문 기자\n우리나라 인구 위기와 관련...,- 우리나라 인구 위기와 관련한 뉴스가 연일 쏟아짐\n- 대통령이 저출산고령사회위원...,지난달 19일 윤석열 대통령이 저출산고령사회위원회 회의를 주재하며 ‘2030년까지 ...
1,2,"사회보험 지속가능성 '제로'…국민 부담, 어디까지 가능할까","(89) 사회보장제도 전망\n게티이미지뱅크\n정부가 국민연금, 건강보험 등 8대 사...","- 정부가 국민연금, 건강보험 등 8대 사회보험과 기초연금, 기초생활보장 등 공공부...",정부가 국민연금 고령화로 복지지출이 급격히 불어나는 가운데 각종 복지제도의 지속가능...
2,3,"현대차, 60세 정년 후 2년간 '계속 고용'","생각하기와 글쓰기\n\n노사, '정년 연장 TF' 꾸려\n내년 상반기 합의안 도출\...",- 노사가 정년 연장 관련 TF를 꾸려 내년 상반기까지 정년 연장 관련 개선 방안을...,현대자동차가 정년(만 60세) 이후에도 생산직 근로자가 원하면 1년 더 근무할 수 ...
3,4,"원가 제때 반영 않는 전기·가스요금, 괜찮은가",일반 가정용 도시가스 도매 요금은 액화천연가스(LNG) 등 원자재 가격을 반영하는 ...,- 공공요금을 올리면 다른 상품이나 서비스 가격도 올라 고물가를 더 자극한다는 현...,한국전력과 한국가스공사가 천문학적 빚더미에 올라 있는 공공요금을 제때 반영하지 않아...
4,5,중앙은행 무용론'…왜 나오는 걸까?,Cover Story\n그래픽=추덕영 한국경제신문 기자\n세계경제의 향방을 결정지을...,- 세계경제의 향방을 결정지을 변수로 미국 중앙은행(Fed)의 기준금리 인하 여부가...,세계경제의 향방을 결정지을 변수로 미국 중앙은행(Fed)의 기준금리 인하 여부가 가...


In [80]:
# 정확성 평가 결과를 저장할 열 추가
edu['accuracy_result'] = edu.apply(lambda row: eval_accuracy(row['Summary_bart'], row['Summary']), axis=1)


In [81]:
edu_result =edu

In [82]:
edu_result

,num,Title,Content,Summary,Summary_bart,accuracy_result
0,1,인구 감소 위기는 청소년 세대의 문제,Cover Story\n그래픽=김선우 한국경제신문 기자\n우리나라 인구 위기와 관련...,- 우리나라 인구 위기와 관련한 뉴스가 연일 쏟아짐\n- 대통령이 저출산고령사회위원...,지난달 19일 윤석열 대통령이 저출산고령사회위원회 회의를 주재하며 ‘2030년까지 ...,정확성 : 4점정확성 : 4점
1,2,"사회보험 지속가능성 '제로'…국민 부담, 어디까지 가능할까","(89) 사회보장제도 전망\n게티이미지뱅크\n정부가 국민연금, 건강보험 등 8대 사...","- 정부가 국민연금, 건강보험 등 8대 사회보험과 기초연금, 기초생활보장 등 공공부...",정부가 국민연금 고령화로 복지지출이 급격히 불어나는 가운데 각종 복지제도의 지속가능...,정확성 : 4정확성 : 4
2,3,"현대차, 60세 정년 후 2년간 '계속 고용'","생각하기와 글쓰기\n\n노사, '정년 연장 TF' 꾸려\n내년 상반기 합의안 도출\...",- 노사가 정년 연장 관련 TF를 꾸려 내년 상반기까지 정년 연장 관련 개선 방안을...,현대자동차가 정년(만 60세) 이후에도 생산직 근로자가 원하면 1년 더 근무할 수 ...,정확성 : 4정확성 : 4
3,4,"원가 제때 반영 않는 전기·가스요금, 괜찮은가",일반 가정용 도시가스 도매 요금은 액화천연가스(LNG) 등 원자재 가격을 반영하는 ...,- 공공요금을 올리면 다른 상품이나 서비스 가격도 올라 고물가를 더 자극한다는 현...,한국전력과 한국가스공사가 천문학적 빚더미에 올라 있는 공공요금을 제때 반영하지 않아...,정확성 : 4점정확성 : 4점
4,5,중앙은행 무용론'…왜 나오는 걸까?,Cover Story\n그래픽=추덕영 한국경제신문 기자\n세계경제의 향방을 결정지을...,- 세계경제의 향방을 결정지을 변수로 미국 중앙은행(Fed)의 기준금리 인하 여부가...,세계경제의 향방을 결정지을 변수로 미국 중앙은행(Fed)의 기준금리 인하 여부가 가...,정확성 : 4정확성 : 4


In [87]:
edu["accuracy_result"] = edu["accuracy_result"].apply(remove_duplicates)
edu

,num,Title,Content,Summary,Summary_bart,accuracy_result
0,1,인구 감소 위기는 청소년 세대의 문제,Cover Story\n그래픽=김선우 한국경제신문 기자\n우리나라 인구 위기와 관련...,- 우리나라 인구 위기와 관련한 뉴스가 연일 쏟아짐\n- 대통령이 저출산고령사회위원...,지난달 19일 윤석열 대통령이 저출산고령사회위원회 회의를 주재하며 ‘2030년까지 ...,정확성 : 4
1,2,"사회보험 지속가능성 '제로'…국민 부담, 어디까지 가능할까","(89) 사회보장제도 전망\n게티이미지뱅크\n정부가 국민연금, 건강보험 등 8대 사...","- 정부가 국민연금, 건강보험 등 8대 사회보험과 기초연금, 기초생활보장 등 공공부...",정부가 국민연금 고령화로 복지지출이 급격히 불어나는 가운데 각종 복지제도의 지속가능...,정확성 : 4
2,3,"현대차, 60세 정년 후 2년간 '계속 고용'","생각하기와 글쓰기\n\n노사, '정년 연장 TF' 꾸려\n내년 상반기 합의안 도출\...",- 노사가 정년 연장 관련 TF를 꾸려 내년 상반기까지 정년 연장 관련 개선 방안을...,현대자동차가 정년(만 60세) 이후에도 생산직 근로자가 원하면 1년 더 근무할 수 ...,정확성 : 4
3,4,"원가 제때 반영 않는 전기·가스요금, 괜찮은가",일반 가정용 도시가스 도매 요금은 액화천연가스(LNG) 등 원자재 가격을 반영하는 ...,- 공공요금을 올리면 다른 상품이나 서비스 가격도 올라 고물가를 더 자극한다는 현...,한국전력과 한국가스공사가 천문학적 빚더미에 올라 있는 공공요금을 제때 반영하지 않아...,정확성 : 4
4,5,중앙은행 무용론'…왜 나오는 걸까?,Cover Story\n그래픽=추덕영 한국경제신문 기자\n세계경제의 향방을 결정지을...,- 세계경제의 향방을 결정지을 변수로 미국 중앙은행(Fed)의 기준금리 인하 여부가...,세계경제의 향방을 결정지을 변수로 미국 중앙은행(Fed)의 기준금리 인하 여부가 가...,정확성 : 4


In [27]:
edu.to_csv("/Users/kimminji/Desktop/eval_result.csv",encoding=)

In [56]:
news = pd.read_csv('/Users/kimminji/Desktop/해외뉴스요약.csv')
news = news[0:3]
news

,date,title,text,title_ko,text_ko
0,2024-07-16 00:00:00,Asia-Pacific markets mixed after Fed Chair Pow...,China's biggest policy meeting in six years ki...,파월 연준 의장의 금리 인하 발언 이후 아시아·태평양 시장이 혼조세를 보이며 월가를...,중국이 6년 만에 최대 규모의 정책회의를 이번 주에 시작한다. Wang Yukun ...
1,2024-07-16 00:00:00,Gold drifts higher as U.S. rate cut optimism b...,"A five hundred gram gold bar, left, and a a on...",미국 금리 인하 낙관론이 매력을 높임에 따라 금이 더 높게 표류하다,스위스 제조업체인 아르고르 헤바우스 SA가 헝가리 부다페스트에서 생산한 500g짜리...
2,2024-07-16 00:00:00,Europe heads for mixed open as markets digest ...,U.S. Federal Reserve Chair Jerome Powell speak...,유럽은 시장이 금리 인하에 대한 파월 의장의 발언을 소화하면서 혼조세로 치닫고 있다,"제롬 파월 미국 연방준비제도 의장이 2022년 6월 15일 금리, 경제 및 통화정책..."


In [57]:
# Content 열의 각 항목을 요약하여 Summary 열에 추가
news['Summary_bart'] = news['text_ko'].apply(summarize_text)

In [58]:
news

,date,title,text,title_ko,text_ko,Summary_bart
0,2024-07-16 00:00:00,Asia-Pacific markets mixed after Fed Chair Pow...,China's biggest policy meeting in six years ki...,파월 연준 의장의 금리 인하 발언 이후 아시아·태평양 시장이 혼조세를 보이며 월가를...,중국이 6년 만에 최대 규모의 정책회의를 이번 주에 시작한다. Wang Yukun ...,중국이 6년 만에 최대 규모의 정책회의를 이번 주에 시작한다. Wang Yukun ...
1,2024-07-16 00:00:00,Gold drifts higher as U.S. rate cut optimism b...,"A five hundred gram gold bar, left, and a a on...",미국 금리 인하 낙관론이 매력을 높임에 따라 금이 더 높게 표류하다,스위스 제조업체인 아르고르 헤바우스 SA가 헝가리 부다페스트에서 생산한 500g짜리...,스위스 제조업체인 아르고르 헤바우스 SA가 헝가리 부다페스트에서 생산한 500g짜리...
2,2024-07-16 00:00:00,Europe heads for mixed open as markets digest ...,U.S. Federal Reserve Chair Jerome Powell speak...,유럽은 시장이 금리 인하에 대한 파월 의장의 발언을 소화하면서 혼조세로 치닫고 있다,"제롬 파월 미국 연방준비제도 의장이 2022년 6월 15일 금리, 경제 및 통화정책...","제롬 파월 미국 연방준비제도 의장이 2022년 6월 15일 금리, 경제 및 통화정책..."


In [59]:
# 정확성 평가 결과를 저장할 열 추가
news['accuracy_result'] = news.apply(lambda row: eval_accuracy(row['Summary_bart'], row['text_ko']), axis=1)

In [60]:
news

,date,title,text,title_ko,text_ko,Summary_bart,accuracy_result
0,2024-07-16 00:00:00,Asia-Pacific markets mixed after Fed Chair Pow...,China's biggest policy meeting in six years ki...,파월 연준 의장의 금리 인하 발언 이후 아시아·태평양 시장이 혼조세를 보이며 월가를...,중국이 6년 만에 최대 규모의 정책회의를 이번 주에 시작한다. Wang Yukun ...,중국이 6년 만에 최대 규모의 정책회의를 이번 주에 시작한다. Wang Yukun ...,정확성 : 5정확성 : 5
1,2024-07-16 00:00:00,Gold drifts higher as U.S. rate cut optimism b...,"A five hundred gram gold bar, left, and a a on...",미국 금리 인하 낙관론이 매력을 높임에 따라 금이 더 높게 표류하다,스위스 제조업체인 아르고르 헤바우스 SA가 헝가리 부다페스트에서 생산한 500g짜리...,스위스 제조업체인 아르고르 헤바우스 SA가 헝가리 부다페스트에서 생산한 500g짜리...,정확성 : 5정확성 : 5
2,2024-07-16 00:00:00,Europe heads for mixed open as markets digest ...,U.S. Federal Reserve Chair Jerome Powell speak...,유럽은 시장이 금리 인하에 대한 파월 의장의 발언을 소화하면서 혼조세로 치닫고 있다,"제롬 파월 미국 연방준비제도 의장이 2022년 6월 15일 금리, 경제 및 통화정책...","제롬 파월 미국 연방준비제도 의장이 2022년 6월 15일 금리, 경제 및 통화정책...",정확성 : 5정확성 : 5


In [85]:
news["accuracy_result"] = news["accuracy_result"].apply(remove_duplicates)


In [61]:
news.to_csv("/Users/kimminji/Desktop/eval_result_news.csv")